# Set up a simple training loop

In [ ]:
from utils import *
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt; plt.style.use('dark_background')
from tqdm import tqdm

In [ ]:
N_SAMPLES = 100
# quantization parameters
EPSI = 0.15 # quantization step
MAX_SIG = 2.5 # maximum value of the signal
NLEVELS = int(2*MAX_SIG / EPSI)//2+1 # number of quantization levels
print(f"Quantization step: {EPSI}, Number of levels: {NLEVELS}")
# training parameters
N_EPOCHS = 10
BATCH_SIZE = 32
LR = 3e-3
LATENT_DIM = 16
N_HIDDEN = 128
# dataset parameters
N_DATASET = 20_000
N_FREQS = 5

In [ ]:
# architecture
class Net(nn.Module):
    def __init__(self, in_dim, out_dim, hidden_dim=128):
        super(Net, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            # nn.ReLU(),
            # nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, out_dim)
        )
    def forward(self, x): return self.fc(x)

In [ ]:
# create a dataset of random signals
class SigDS(Dataset):
    def __init__(self, n_ds):
        self.n_ds = n_ds
        self.data = th.stack([create_random_signal(N_SAMPLES, N_FREQS) for _ in range(n_ds)])
    def __len__(self): return self.n_ds
    def __getitem__(self, idx): return self.data[idx]

ds = SigDS(N_DATASET)
dl = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
# plot 10 random signals
plt.figure(figsize=(10, 3))
for i in range(10):
    plt.subplot(5, 2, i+1)
    plt.plot(ds[i])
    plt.xticks([]), plt.yticks([])
plt.tight_layout(), plt.show();

In [ ]:
# training loop
enc = Net(N_SAMPLES, LATENT_DIM, N_HIDDEN)
dec = Net(LATENT_DIM, N_SAMPLES, N_HIDDEN)
opt = optim.Adam(list(enc.parameters()) + list(dec.parameters()), lr=LR)

mse_loss = nn.MSELoss()
l1_loss = nn.L1Loss()
# H_loss = HLoss1()
# H_loss = HLoss2(EPSI, MAX_SIG)
H_loss = HLoss3(EPSI, MAX_SIG, temperature=0.1)

lmses, lL1s, lHs = [], [], []
for epoch in (range(N_EPOCHS)):
    elmse, elL1, elH = 0, 0, 0
    for x in dl:
        opt.zero_grad()

        z = enc(x)
        x̂ = dec(z)

        lmse = mse_loss(x̂, x)
        lL1 = l1_loss(x̂, x)
        lH = H_loss(x̂, x)

        elmse += lmse.item()
        elL1 += lL1.item()
        elH += lH.item()

        # loss = lmse
        # loss = lL1
        # loss = lH 
        loss = lH + 0.1*lmse
        loss.backward()

        opt.step()

    lmses.append(elmse/len(dl))
    lL1s.append(elL1/len(dl))
    lHs.append(elH/len(dl))

    print(f'ep {epoch} -> mse:{lmses[-1]:.4f}, L1:{lL1s[-1]:.4f}, H:{lHs[-1]:.4f}')

In [ ]:
# plot losses
lmses, lL1s, lHs =  th.tensor(lmses), th.tensor(lL1s), th.tensor(lHs)
lmses = lmses/th.max(lmses)
lL1s = lL1s/th.max(lL1s)
lHs = lHs/th.max(lHs)
plt.figure(figsize=(10, 4))
plt.subplot(3,1, 1)
plt.plot(lmses, label='mse')
plt.yscale('log')
plt.legend()
plt.subplot(3,1, 2)
plt.plot(lL1s, label='L1')
plt.yscale('log')
plt.legend()
plt.subplot(3,1, 3)
plt.plot(lHs, label='H')
plt.yscale('log')
plt.legend()
plt.show()
print(f'train losses: mse:{lmses[-1]:.4f}, L1:{lL1s[-1]:.4f}, H:{lHs[-1]:.4f}')


# evaluate on unseen data useing mse loss
test_in = SigDS(100).data
test_out = dec(enc(SigDS(100).data))
test_loss = mse_loss(test_out, test_in)
print(f'test loss: {test_loss}')

In [ ]:
# compare input and output
plt.figure(figsize=(10, 10))
NPLOTS = 40
err = th.zeros(NPLOTS, 3)
for i in range(NPLOTS):
    plt.subplot(10, 4, i+1)
    x = create_random_signal(N_SAMPLES).view(1, N_SAMPLES)
    x̂ = dec(enc(x))
    err[i] = th.tensor([mse_loss(x̂, x), l1_loss(x̂, x), H_loss(x̂, x)])
    x, x̂ = x.view(-1).detach(), x̂.view(-1).detach()
    plt.plot(x, label='input')
    plt.plot(x̂, label='output')
    plt.xticks([]), plt.yticks([])
plt.suptitle(f'mse:{err[:,0].mean():.4f},  L1:{err[:,1].mean():.4f},  H:{err[:,2].mean():.4f}')
plt.tight_layout()
plt.legend()
plt.show()